In [9]:
import numpy as np
import pandas as pd
import datetime as dt
import logging
import os
from pathlib import Path
import io
import json
import zipfile
import requests

In [10]:
def get_qualtrics_survey(use_labels=True):
    """Export Qualtrics Survey Responses via API Call According to the Documentation.

    Input
    :param survey_id: Qualtrics ID for Survey to export.
    """
    surveyId = "SV_8iwwlvdP2dqQWZU"
    fileFormat = "csv"
    dataCenter = "nielsenscarborough.iad1"
    apiToken = "gZRjDAnNzFDDO7kuV8HDXI9GPG2xMg1rUhDOsu5I"

    # Setting static parameters
    requestCheckProgress = 0.0
    progressStatus = "inProgress"
    baseUrl = "https://{0}.qualtrics.com/API/v3/surveys/{1}/export-responses/".format(
        dataCenter, surveyId
    )
    headers = {
        "content-type": "application/json",
        "x-api-token": apiToken,
    }

    # Step 1: Creating Data Export
    downloadRequestUrl = baseUrl
    downloadRequestPayload = {"format": fileFormat, "useLabels": use_labels}

    downloadRequestPayload = json.dumps(downloadRequestPayload)

    downloadRequestResponse = requests.request(
        "POST", downloadRequestUrl, data=downloadRequestPayload, headers=headers
    )
    progressId = downloadRequestResponse.json()["result"]["progressId"]
    print(downloadRequestResponse.text)

    # Step 2: Checking on Data Export Progress and waiting until export is ready
    while progressStatus != "complete" and progressStatus != "failed":
        print("progressStatus = " + progressStatus)
        requestCheckUrl = baseUrl + progressId
        requestCheckResponse = requests.request("GET", requestCheckUrl, headers=headers)
        requestCheckProgress = requestCheckResponse.json()["result"]["percentComplete"]
        print("Download is " + str(requestCheckProgress) + " complete")
        progressStatus = requestCheckResponse.json()["result"]["status"]

    # step 2.1: Check for error
    if progressStatus == "failed":
        raise Exception("export failed")

    fileId = requestCheckResponse.json()["result"]["fileId"]

    # Step 3: Downloading file
    requestDownloadUrl = baseUrl + fileId + "/file"
    requestDownload = requests.request(
        "GET", requestDownloadUrl, headers=headers, stream=True
    )

    # Step 4: Unzipping the file and read as pandas dataframe
    print("Reading zip to pandas")
    r_zip = zipfile.ZipFile(io.BytesIO(requestDownload.content))
    df = pd.read_csv(r_zip.open(r_zip.namelist()[0]), sep=",", dtype=str)
    print("Successfully read from API into Pandas dataframe")
    return df

In [11]:
df=get_qualtrics_survey()

{"result":{"progressId":"ES_3ldSewIWzsqCKR8","percentComplete":0.0,"status":"inProgress"},"meta":{"requestId":"29f7a3e1-f93b-4d33-9e99-ced31b645872","httpStatus":"200 - OK"}}
progressStatus = inProgress
Download is 0.0 complete
progressStatus = inProgress
Download is 0.0 complete
progressStatus = inProgress
Download is 0.0 complete
progressStatus = inProgress
Download is 0.0 complete
progressStatus = inProgress
Download is 0.0 complete
progressStatus = inProgress
Download is 100.0 complete
Reading zip to pandas
Successfully read from API into Pandas dataframe


In [12]:
df.drop([0,1], axis=0, inplace=True)

In [13]:
qualtrics_df=df

In [14]:
df.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'age_nielsen_employee',
       'address_confirm ', 'address_update_1', 'address_update_2',
       'address_update_3', 'address_update_4', 'address_update_5',
       'hh_size_confirm', 'hh_size_update', 'address_1', 'address_2', 'city',
       'hh_size', 'state', 'zip'],
      dtype='object')

In [15]:
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,address_update_4,address_update_5,hh_size_confirm,hh_size_update,address_1,address_2,city,hh_size,state,zip
2,2023-08-21 12:51:30,2023-08-21 12:51:54,Survey Preview,NaN,100,23,True,2023-08-21 12:51:55,R_12audmPhySxTiHP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-21 12:54:14,2023-08-21 12:54:33,IP Address,100.16.232.120,100,18,True,2023-08-21 12:54:34,R_3DvVoOH8VsBV2SG,Neuhoff,...,NaN,NaN,"Yes, this is correct",NaN,7055 melting shadows lane,NaN,Columbia,2,MD,21045
4,2023-08-21 13:24:05,2023-08-21 13:25:06,IP Address,96.244.131.212,100,60,True,2023-08-21 13:25:07,R_3Mum6Cw7GTlbcmI,Pierce,...,NaN,NaN,"Yes, this is correct",NaN,6166 Commadore Ct,NaN,Columbia,2,MD,21045
5,2023-08-21 13:25:52,2023-08-21 13:26:20,IP Address,71.244.153.167,100,28,True,2023-08-21 13:26:21,R_3GePp1f1oZKZWZF,Gentry,...,NaN,NaN,"Yes, this is correct",NaN,8337 Montgomery Run Rd.,Apt. E,Ellicott City,1,MD,21043
6,2023-08-21 13:28:42,2023-08-21 13:29:11,IP Address,76.136.231.104,100,29,True,2023-08-21 13:29:12,R_WDJO7ddiHLKTeKt,Solano,...,NaN,NaN,"Yes, this is correct",NaN,1011 N Harvard Ave,NaN,Arlington Heights,2,IL,60004
